2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


プーリング層なども作成することで、CNNの基本形を完成させます。クラスの名前はScratch2dCNNClassifierとしてください。

### データセットの用意

引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。


今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。


`(n_samples, n_channels, height, width)`の`NCHW`または`(n_samples, height, width, n_channels)`の`NHWC`どちらかの形にしてください。

In [10]:
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

/Users/SHOTA/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/SHOTA/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/SHOTA/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/SHOTA/.py

#### 処理の流れ

- **フォワード**

1. 入力データ
2. im2colで入力データを2次元配列に展開
3. 

### 【問題2】2次元畳み込み後の出力サイズ

畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。

$$
N_{h,out} =  \frac{N_{h,in}+2P_{h}-F_{h}}{S_{h}} + 1\\
N_{w,out} =  \frac{N_{w,in}+2P_{w}-F_{w}}{S_{w}} + 1
$$

$N_{out}$ : 出力のサイズ（特徴量の数）


$N_{in}$ : 入力のサイズ（特徴量の数）


$P$ : ある方向へのパディングの数


$F$ : フィルタのサイズ


$S$ : ストライドのサイズ


$h$ が高さ方向、 $w$ が幅方向である

In [11]:
def after_conv_outsize(N, P, F, S):
    return (N + 2*P - F) // S + 1

### im2colの実装

In [12]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    input_data : (データ数, チャンネル, 高さ, 幅)の4次元配列からなる入力データ
    filter_h : フィルターの高さ
    filter_w : フィルターの幅
    stride : ストライド
    pad : パディング
    Returns
    -------
    col : 2次元配列
    """
    # input_dataから、バッチサイズN, チャンネル数C, 画像高さH, 画像幅W を取得
    N, C, H, W = input_data.shape
    
    # 畳み込み処理後の出力の高さ out_h, 幅 out_w を計算
    out_h = after_conv_outsize(H, pad, FH, stride)
    out_w = after_conv_outsize(W, pad, FW, stride)

    # 画像のパディング(高さ方向 pad_h, 幅方向 pad_w)
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    
    # 計算結果を保存するためのゼロ行列の作成
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        # 縦方向のスライシング範囲の最大値y_maxを求める
        y_max = y + stride*out_h
        for x in range(filter_w):
            # 横方向のスライシング範囲の最大値x_maxを求める
            x_max = x + stride*out_w
            # スライシング結果をゼロ行列に格納
            # y から y_max まで stride_h 間隔でスライシング
            # x から x_max まで stride_w 間隔でスライシング
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    
    #(0:N, 1:C, 2:filter_h, 3:filter_w, 4:out_h, 5:out_w)
    #軸の入れ替え
    #(0:N, 4:out_h, 5:out_w, 1:C, 2:filter_h, 3:filter_w)
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


- 四次元配列のnp.pad

４次元配列 img (ミニバッチサイズ、チャンネル数、縦幅、横幅)のパディング処理をする場合を考えてみます。縦方向のパディングを p_h, 横方向のパディングを p_w とします。バッチ方向やチャンネル方向のパディングは不要なので、

`img = np.pad(img, [(0, 0), (0, 0), (p_h, p_h), (p_w, p_w)], 'constant')`

In [15]:
img = np.array([[[[1, 2, 3], [4, 5, 6]], [[7, 8, 9],[0, 1, 2]]]])
print(img)
print('img.shape = ', img.shape)
img = np.pad(img, [(0,0), (0,0), (1,1), (1,1)], 'constant')
print(img)

[[[[1 2 3]
   [4 5 6]]

  [[7 8 9]
   [0 1 2]]]]
img.shape =  (1, 2, 2, 3)
[[[[0 0 0 0 0]
   [0 1 2 3 0]
   [0 4 5 6 0]
   [0 0 0 0 0]]

  [[0 0 0 0 0]
   [0 7 8 9 0]
   [0 0 1 2 0]
   [0 0 0 0 0]]]]


### col2imの実装

In [ ]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """
    Parameters
    ----------
    col :
    input_shape : 入力データの形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad
    Returns
    -------
    """
    N, C, H, W = input_shape
    out_h = after_conv_outsize(H, pad, FH, stride)
    out_w = after_conv_outsize(W, pad, FW, stride)
    
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)
    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]


In [22]:
class Conv2d:
    def __init__(self, W, B, stride=1, pad=0, Ir=1):
        self.W = W
        self.B = B
        self.pad = pad
        self.stride = stride
        self.F = self.W.shape[-1]
    
    def forward(self, X):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = X.shape
        out_h = after_conv_outsize(H, self.pad, FH, self.stride)
        out_h = after_conv_outsize(H, self.pad, FH, self.stride)
        
        self.col = im2col(X, FH, FW, self.stride)
        self.col_W = self.W.reshape(FN, -1).T
        print("col:", self.col.shape)
        print("col_W:", self.col_W)
        
        A = np.dot(self.col, self.col_W) + self.B
        A = A.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        return A
    
    
    def backward(self, dA):
        FN, C, FH, FW = self.W.shape
        
        dA = A.reshape(0, 2, 3, 1).reshape(-1, FN)
        
        self.dB = np.sum(dA, axis=0)
        self.dW = np.dot(self.col.T, dA)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
        
        dcol = np.dot(dA, self.col_W.T)
        dx = col2im(dcol, self.X.shape, FH, FW, self.stride, sel.pad)
        
        return dx